In [56]:
import transformers 
import datasets
import numpy as np
import pandas as pd
import torch

In [61]:
import os
from datasets import Dataset, DatasetDict, Image
from torch.utils.data import Dataset as Dataset_torch

def list_png_files(root_folder, extension):
    png_files = []
    for folder, _, files in os.walk(root_folder):
        if "test" not in folder.lower():
            for file in files:
                if file.lower().endswith(extension) and "_inst" not in file.lower():
                    png_files.append(os.path.join(folder, file))
    return png_files


# Example usage:
root_directory = 'data/idd20k_lite'
png_files_list = list_png_files(root_directory,'.png')
jpg_files_list = list_png_files(root_directory,'.jpg')
print(png_files_list)
print(jpg_files_list)
print("Count of Images: {}".format(len(jpg_files_list)))
print("Count of Masks: {}".format(len(png_files_list)))

def create_dataset(image_paths, label_paths):
    dataset = Dataset.from_dict({"image": sorted(image_paths),
                                "label": sorted(label_paths)})
    dataset = dataset.cast_column("image", Image())
    dataset = dataset.cast_column("label", Image())
    return dataset

# step 1: create Dataset objects
train_dataset = create_dataset(jpg_files_list, png_files_list)


['data/idd20k_lite/val/181/016719_label.png', 'data/idd20k_lite/val/181/013948_label.png', 'data/idd20k_lite/val/181/001500_label.png', 'data/idd20k_lite/val/181/016119_label.png', 'data/idd20k_lite/val/379/frame46571_label.png', 'data/idd20k_lite/val/379/frame80048_label.png', 'data/idd20k_lite/val/379/frame37118_label.png', 'data/idd20k_lite/val/379/frame28338_label.png', 'data/idd20k_lite/val/379/frame21797_label.png', 'data/idd20k_lite/val/379/frame45784_label.png', 'data/idd20k_lite/val/379/frame9094_label.png', 'data/idd20k_lite/val/379/frame23283_label.png', 'data/idd20k_lite/val/205/frame5153_label.png', 'data/idd20k_lite/val/205/frame5783_label.png', 'data/idd20k_lite/val/533/frame5719_label.png', 'data/idd20k_lite/val/533/frame0079_label.png', 'data/idd20k_lite/val/272/frame0144_label.png', 'data/idd20k_lite/val/172/874002_label.png', 'data/idd20k_lite/val/172/894999_label.png', 'data/idd20k_lite/val/172/178400_label.png', 'data/idd20k_lite/val/172/164487_label.png', 'data/id

In [47]:
# step 2: create DatasetDict
dataset = DatasetDict({
     "train": train_dataset
     }
)
np.unique(np.array(dataset["train"][10]["label"]))

array([  0,   2,   3,   4,   5,   6, 255], dtype=uint8)

## Dataloader



In [67]:
class idd(Dataset_torch):
    def __init__(self, img_list, png_list, transforms=None):
        super(idd, self).__init__()
        self.img_list = img_list
        self.png_list = png_list
        self.augmentations = augmentations

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        
        img = self.img_list[idx]
        mask = self.png_list[idx]

        if self.transforms:
			img = self.transforms(image)   # apply any transformation

        return img, mask

train_dataset = idd(img_list=jpg_list_files, png_list=png_list_files, transforms=None)

train_loader = DataLoader(
    train_dataset, batch_size=bs, shuffle=True, num_workers=num_workers
)

TabError: inconsistent use of tabs and spaces in indentation (4022087747.py, line 16)

In [59]:
class IDD_Lite20K(DatasetDict):
	"""Image (semantic) segmentation dataset."""
	def __init__(self, dataset, transforms = None):
        # Variables for the dataset
		self.dataset = dataset
		self.transforms = transforms
		
		# we will read the images and labels and store them in these lists.
		self.images = [] # -> list
		self.labels = [] # -> list

	def __len__(self):
		# this returns and only returns the length of the dataset
		return len(self.images)

	def __getitem__(self, idx):

		if torch.is_tensor(idx):
			idx = idx.tolist()

        # retrieve the image from the list created of the dataset
		image = self.images[idx]
		#image = Image.fromarray(image) # -> return it as an image

		label = self.labels[idx]  # -> get the labels of the image

		if self.transforms:
			image = self.transforms(image)   # apply any transformation

		return image, label  # -> return the image and the labels as a tuple.

dataloader = IDD_Lite20K(dataset=dataset, transforms=None)
print(type(dataloader))

print("Number of training examples:", len(dataloader))

fig = plt.figure()

for i, sample in enumerate(face_dataset):
    print(i, sample['image'].shape, sample['landmarks'].shape)

    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    show_landmarks(**sample)

    if i == 3:
        plt.show()
        break


<class '__main__.IDD_Lite20K'>
Number of training examples: 0
